In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [3]:
athlete_info_palbar = pd.read_csv('athlete_events.csv', sep=',')

In [4]:
sport_participants_palbar = athlete_info_palbar.groupby(['Season', 'Sport']).agg({'ID':'nunique'}).reset_index()
sport_participants_palbar = sport_participants_palbar.rename(columns={'ID': 'Participants'})

In [5]:
year_sport_participants_palbar = athlete_info_palbar.groupby(['Year','Season','Sport']).agg({'ID':'nunique'}).reset_index()
year_sport_participants_palbar = year_sport_participants_palbar.rename(columns={'ID': 'Participants'})

In [6]:
season_option_list_palbar = []
season_list_palbar = sport_participants_palbar['Season'].unique()
for i in season_list_palbar:
    season_option_list_palbar.append({'label':i, 'value':i})

In [7]:
year_option_list_palbar = []
year_list_palbar = np.sort(athlete_info_palbar["Year"].unique())
for i in year_list_palbar:
    year_option_list_palbar.append({'label':i, 'value':i})

In [27]:
app = dash.Dash()
app.layout = html.Div([    
    html.Div('Olympic Season Top Ten Sports with the Highest Numbers of Participants'),
    html.Br(),
    html.Label('Note: Winter and Summer Olympic were held in the same year up until 1992. After 1992, Olympic Games occured on a four year cycle starting with Winter in 1994, then Summer in 1996, then Winter in 1998, and so on.'),
    html.Br(),
    html.Br(),
    html.Label('Select the Olympic season to see the top ten sports with the highest numbers of athletes participantion'),
    html.Br(),
    html.Br(),
    html.Label('Select Season:', style = {'width':'50%', 'display':'inline-block'}),
    
    html.Label('Select Year of the Season:', style = {'text-align':'left', 'width':'50%', 'display':'inline-block'}),
    
    html.Div(dcc.Dropdown(id = 'dropdown_season_palbar', options = season_option_list_palbar, value = 'Summer'),
             style = {'text-align':'center', 'width':'30%', 'display':'inline-block'}
            ),
    
    html.Div(dcc.Dropdown(id = 'dropdown_year_palbar', options = year_option_list_palbar, value = 1992),
             style = {'text-align':'center', 'width':'30%', 'display':'inline-block', 'margin-left':'300px'}
            ),
    html.Br(),
    
    html.Div([dcc.Graph(id = 'plot_sport_graph_palbar', figure = {})], style = {'display':'inline-block'}),
    html.Div([dcc.Graph(id = 'plot_year_graph_palbar', figure = {})], style = {'display':'inline-block'}),
                    ])

@app.callback([dash.dependencies.Output('plot_sport_graph_palbar', 'figure'),
               dash.dependencies.Output('plot_year_graph_palbar', 'figure')],
              [dash.dependencies.Input('dropdown_season_palbar', 'value'),
               dash.dependencies.Input('dropdown_year_palbar', 'value')]
             )

def update_graph(dropdown_season_palbar,dropdown_year_palbar):
    season_sport_participants_palbar = sport_participants_palbar.copy()
    season_sport_participants_palbar = season_sport_participants_palbar[season_sport_participants_palbar['Season'] == dropdown_season_palbar]
    
    sorted_sport_participants_palbar = season_sport_participants_palbar.sort_values('Participants',ascending = False)
    top_ten_participants_palbar = sorted_sport_participants_palbar[:10]
    sport_fig_palbar = px.bar(top_ten_participants_palbar.sort_values('Participants',ascending=True), x='Participants', y='Sport',
                       color_discrete_sequence =['lightblue']*len(top_ten_participants_palbar))
    
    year_sport_palbar = year_sport_participants_palbar.copy()
    year_sport_palbar = year_sport_palbar[year_sport_palbar['Season'] == dropdown_season_palbar]
    
    year_participants_palbar = year_sport_palbar[year_sport_palbar['Year'] == dropdown_year_palbar]
    sorted_year_palbar = year_participants_palbar.sort_values('Participants',ascending = False)
    top_ten_year_palbar = sorted_year_palbar[:10]
    year_fig_palbar = px.bar(top_ten_year_palbar.sort_values('Participants',ascending=True), x='Participants', y='Sport',
                       color_discrete_sequence =['lightblue']*len(top_ten_participants_palbar))
    
    sport_fig_palbar.update_layout(
        width = 650,
        height = 500,
        title = '<b>Top Ten Sports In Olympic {} Season</b>'.format(dropdown_season_palbar),
        xaxis_tickangle = -45,
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        xaxis = dict(
            title = '<b>Total Number of Participants</b>',
            title_font = {'size': 14},
            tickmode = 'linear',
            tick0 = 0,
            dtick = 2000,
            range = [0, 23000],
            gridwidth = 1,
            gridcolor = 'lightblue',
        ),
        yaxis = dict(
            title = '<b>Sport</b>',
            title_font = {'size': 14}
        )
    ),
    
    year_fig_palbar.update_layout(
        width = 650,
        height = 500,
        title = '<b>Top Ten Sports In Olympic {} Season Year {}</b>'.format(dropdown_season_palbar,dropdown_year_palbar),
        xaxis_tickangle = -45,
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        xaxis = dict(
            title = '<b>Total Number of Participants</b>',
            title_font = {'size': 14},
            tickmode = 'linear',
            tick0 = 0,
            dtick = 200,
            range = [0, 2300],
            gridwidth = 1,
            gridcolor = 'lightblue'
        ),
        yaxis = dict(
            title = '<b>Sport</b>',
            title_font = {'size': 14}
        )
    )    
    return sport_fig_palbar,year_fig_palbar

if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Dec/2020 14:38:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2020 14:38:04] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2020 14:38:04] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2020 14:38:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2020 14:38:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2020 14:38:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2020 14:38:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2020 14:38:19] "POST /_dash-update-component HTTP/1.1" 200 -
